<a href="https://colab.research.google.com/github/Akshaypakhle10/ML/blob/master/Multitouch_Attribution_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Probabilistic Multi-Touch Attribution Model

We will demonstrate how to apply a probabilistic, multi-touch attribution model to your GA data using Python and BigQuery libraries. The procedure in this notebook is based on an article published in 2011 by [Xuhui Shao.](https://www.linkedin.com/in/xuhuishao/) The resulting score helps marketers better understand their channel investments.

Big thanks to [Anže Kravanja](https://www.linkedin.com/in/an%C5%BEe-kravanja-54006194/), [Luka Cempre](https://www.linkedin.com/in/luka-cempre-9b5ba71a/), and [Clay Porter](https://www.linkedin.com/in/clayporter1/) for their help in putting this notebook together!

Here is the author, [Xuhui Shao](https://www.linkedin.com/in/xuhuishao/), describing the <a href="https://www.clickz.com/the-math-behind-multi-touch-attribution/50028/">math behind the model</a>:  

<img src="https://storage.googleapis.com/pats-test/images/model.png">

>Let’s say for one converted user, email contributes 15 percent alone and makes other channels such as display and video 20 percent better. The overall contribution of email to this conversion is 35 percent.

>The contributions of all the touch points are then normalized to ensure they always add up to 100 percent. In other words, the credit of the conversion is proportionately assigned to each touch point based on its relative contribution in increasing the user’s probability to convert.

#Markov Chain Approach

In [0]:
#Importing necessary Libraries
import pandas as pd
import numpy as np
import itertools
import urllib2
import matplotlib.pylab as plt
plt.style.use("fivethirtyeight")
import seaborn as sns
import subprocess

## Query GA Data in BigQuery
The <a href="https://storage.googleapis.com/pats-test/sql/multi_touch_attribution.sql">multi_touch_attribution.sql</a> SQL is used to format our data in the appropriate schema for our data pipeline.

You'll want to download and inspect the SQL, and then upload that file to your local Drive directory below:

You're ready to execute the query after adding a target table and start and end dates. In our case we're using the BigQuery GA data for Google Merchandize Store.


In [0]:
#@title Enter your BigQuery project ID { run: "auto", display-mode: "form" }
project = "multi-touch-attribution-268621" #@param {type:"string"}
target = "bigquery-public-data.google_analytics_sample.ga_sessions_*" #@param {type:"string"}
start = "2017-07-01" #@param {type:"date"}
end = "2020-08-01" #@param {type:"date"}


In [0]:
sql_query = urllib2 \
  .urlopen("https://storage.googleapis.com/akshay-pakhle/sql/multi_touch_attribution.sql") \
  .read().format(target,start.replace("-",""),end.replace("-",""))

ga_data = pd.io.gbq.read_gbq(sql_query,project_id=project,dialect='standard')

print ga_data.shape
ga_data.sample(5)

In [10]:
#Reading in Data
df = pd.read_csv('/content/drive/My Drive/Datasets/Channel_attribution.csv')
df.head()

,R05A.01,R05A.02,R05A.03,R05A.04,R05A.05,R05A.06,R05A.07,R05A.08,R05A.09,R05A.10,R05A.11,R05A.12,R05A.13,R05A.14,R05A.15,R05A.16,R05A.17,R05A.18,R05A.19,R05A.20,Output
0,16,4,3.0,5.0,10.0,8.0,6.0,8.0,13.0,20.0,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,9.0,10.0,1.0,4.0,3.0,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9,13,20.0,16.0,15.0,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8,15,20.0,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,16,9,13.0,20.0,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The dataset is structured by having engagement activities as columns and the rows being the channels that were engaged with, in chronological order. In this case, each marketing channel is assigned a fixed numbered value which is then displayed in a column n if the n’th engagement from a given user was with that marketing channel. Channel 21 is a conversion and our dataset only contains records of converting user journeys.

In [0]:
###Cleaning up
# Grab list of columns to iterate through
cols = df.columns

# Iterate through columns to change all ints to str and remove any trailing '.0'
for col in cols:
    df[col] = df[col].astype(str)
    df[col] = df[col].map(lambda x: str(x)[:-2] if '.' in x else str(x))

The Markov chain framework wants the user journeys in a single variable and on the form Channel 1 > Channel 2 > Channel 3 > …, so the next loop creates exactly that

In [13]:
# Create a total path variable
df['Path'] = ''
for i in df.index:
    #df.at[i, 'Path'] = 'Start'
    for x in cols:
        df.at[i, 'Path'] = df.at[i, 'Path'] + df.at[i, x] + ' > '

#Let's see how it looks
df['Path'].head() 

0    16 > 4 > 3 > 5 > 10 > 8 > 6 > 8 > 13 > 20 > 21...
1    2 > 1 > 9 > 10 > 1 > 4 > 3 > 21 > nan > nan > ...
2    9 > 13 > 20 > 16 > 15 > 21 > nan > nan > nan >...
3    8 > 15 > 20 > 21 > nan > nan > nan > nan > nan...
4    16 > 9 > 13 > 20 > 21 > nan > nan > nan > nan ...
Name: Path, dtype: object

This path represents the entire journey of the customer across multiple channels until conversion. These represent various states in our Markov chain.

## Individual Channel Probability

<img src="https://storage.googleapis.com/pats-test/images/channel_probability.jpg">  

    P(y|xi) = positives / positives + negatives = conversions / users = channel conversion rate  


In [0]:
channel_metrics_map = {}  

for user in ga_data.to_records(): 
    
    for channel in user["channels"]:
        
        if not channel in channel_metrics_map:
            channel_metrics_map[channel] = { "conversions": 0.0, "users": 0.0 }
            
        channel_metrics_map[channel]["conversions"] += float(user["converted"])
        channel_metrics_map[channel]["users"] += 1.0 

pd.DataFrame(channel_metrics_map).transpose().sort_values(by='conversions',ascending=False)

## Pairwise Channel Probabilities

<img src="https://storage.googleapis.com/pats-test/images/pairwise_probability.png">

In [0]:
channel_metrics = channel_metrics_map.copy()

for xi,xj in itertools.combinations(channel_metrics.keys(), 2):
    
    key = "{}|{}".format(xi, xj)
    
    for user in ga_data.loc[ga_data.channels.map(lambda c: ((xi in c) & (xj in c))) == True].to_records():
         
        if not key in channel_metrics: 
            channel_metrics[key] = { "conversions": 0.0, "users": 0.0 }

        channel_metrics[key]["conversions"] += float(user[3]) 
        channel_metrics[key]["users"] += 1.0 

pd.DataFrame(channel_metrics).transpose().sort_values(by='conversions', ascending=False) \
    .reset_index().rename(columns={'index':'channels'})[:10]

### Add calculated field: `Conversion Rate`

In [0]:
for key in channel_metrics:
    channel_metrics[key]["conversion rate"] = np.divide(
        channel_metrics[key]["conversions"] , 
        channel_metrics[key]["users"]
    )

pd.DataFrame(channel_metrics).transpose().sort_values(by='conversions', ascending=False) \
    .reset_index().rename(columns={'index':'channels'})[:5]

## Simple Multi-Touch Probabilistic Model
<img src="https://storage.googleapis.com/pats-test/images/model.png">

In [0]:
channels = [ c for c in channel_metrics.keys() if not "|" in c ]
channel_contrib = {} 

for channel in channels:
    
    p_xi = channel_metrics[channel]["conversion rate"]
    
    combinations = [ c for c in channel_metrics.keys() if channel in c and "|" in c ] 
    n = np.divide(1.0 , float(2 * len(combinations)))
    
    sum_part = []
    
    for combination in combinations:
        
        xi_xj = channel_metrics[combination]["conversion rate"]
                                             
        xi = channel_metrics[ combination.split("|")[0] ]["conversion rate"]
        xj = channel_metrics[ combination.split("|")[1] ]["conversion rate"]
                                             
        sum_part.append(xi_xj - xi - xj)
    
    s = np.sum(sum_part)
    channel_contrib[channel] = p_xi + n * (s if s > 0 else 0)

pd.DataFrame([channel_contrib]).transpose()

### Covert to `% of Total`, column sums to 1

In [0]:
c_sum = np.sum([ channel_contrib[ch] for ch in channel_contrib.keys() ])
for key in channel_contrib.keys():
    channel_contrib[key] /= c_sum
    
results = pd.DataFrame(channel_contrib.items())
results.columns=['channel','contribution']
print "The contribution column sums to {}".format(sum(results["contribution"]))
results.sort_values(by='contribution',ascending=False).reset_index(drop=True)

### Add context by creating weighted fields

In [0]:
results["conversions"] = results.channel.map(lambda x: channel_metrics[x]["conversions"])

results["score"] = results["conversions"] * results["contribution"]

results['importance'] = np.divide(
    results['score']-results['score'].min() , 
    results['score'].max()-results['score'].min()
)

results = results.sort_values('importance',ascending=False).fillna(0).reset_index(drop=True)
results

## Visualize the data  

In [0]:
fig, ax = plt.subplots()
cmap = plt.cm.get_cmap('tab20')
for i,row in results.iterrows():
  ax.scatter("contribution", "conversions",label=row.channel,
             s=row.importance*500, c=cmap(i), data=row)
plt.xlabel("Contribution Score")
plt.ylabel("Conversions")
plt.title("Contribution Score by Conversions per Channel")
plt.legend(loc=0)
plt.show()

<h3>Contribution Score over 12 weeks (example):</h3>

In [0]:
from IPython.display import YouTubeVideo
YouTubeVideo("OBZILfAf4xQ",width=768, height=432)